In [18]:

#Install required libraries
import pandas as pd
from gen3.submission import Gen3Submission
from gen3.auth import Gen3Auth
from gen3.query import Gen3Query

In [19]:
### download and import some custom Python scripts from https://github.com/cgmeyer/gen3sdk-python
# import os
#os.system("wget https://raw.githubusercontent.com/cgmeyer/gen3sdk-python/master/expansion/expansion.py")


In [20]:
from expansion import Gen3Expansion

In [21]:
# Initiate instances of the Gen3 SDK Classes using credentials file downloaded from https://staging.midrc.org/identity
# You can view the SDK code/functions in GitHub: https://github.com/uc-cdis/gen3sdk-python
api = 'https://qa-midrc.planx-pla.net/'
cred = 'midrc_api_key.json'
auth = Gen3Auth(api, refresh_file=cred) # authentication class
sub = Gen3Submission(api, auth) # submission class
query = Gen3Query(auth) # query class
exp = Gen3Expansion(api,auth,sub) # class with some custom scripts
exp.get_project_ids()

Getting all project_ids you have access to in https://qa-midrc.planx-pla.net/
['jnkns-jenkins', 'jnkns-load_test', 'jnkns-test.123', 'jnkns-test123']


['jnkns-jenkins', 'jnkns-load_test', 'jnkns-test.123', 'jnkns-test123']

In [34]:
img_stud = exp.get_node_tsvs(projects = 'jnkns-jenkins', node='imaging_study', overwrite=True)


Output written to file: node_tsvs/imaging_study_tsvs/jnkns-jenkins_imaging_study.tsv
node_tsvs/imaging_study_tsvs/jnkns-jenkins_imaging_study.tsv has 8914 records.
length of all dfs: 8914
Master node TSV with 8914 total records written to master_imaging_study.tsv.


In [ ]:
#increasing the volume of the number of records by 10 fold for testing purposes
img_stud_list = [img_stud.copy(deep=True) for i in range(10)]
for i in range(10):
    img_stud_list[i]["submitter_id"] = img_stud['submitter_id']+"_"+str(i) #changing the submitter_id to make them unique

img_stud = pd.concat(img_stud_list)


type
submitter_id	
datasets
cases

In [ ]:
# Trim down the number of records to 4000 to save time on submission
img_stud = img_stud.head(4000)
len(img_stud)

4000

In [36]:
img_stud = img_stud[['type', 'submitter_id', 'datasets.submitter_id', 'cases.submitter_id']]

img_stud['datasets.submitter_id'] = 'dataset_backpedalled_usurers'


img_stud.to_csv('test_img_stud.tsv', sep='\t', index=False)



In [37]:
import time
start = time.time()
sub_test = sub.submit_file(project_id = 'jnkns-jenkins', filename = 'test_img_stud.tsv', chunk_size=1000)
end = time.time()
time_taken = end - start



Submitting test_img_stud.tsv with 4000 records.
Chunk 1 (chunk size: 1000, submitted: 0 of 4000)
	 Succeeded: 1000 entities.
Chunk 2 (chunk size: 1000, submitted: 1000 of 4000)
	 Succeeded: 1000 entities.
Chunk 3 (chunk size: 1000, submitted: 2000 of 4000)
	 Succeeded: 1000 entities.
Chunk 4 (chunk size: 1000, submitted: 3000 of 4000)
	 Succeeded: 1000 entities.
Chunk 5 (chunk size: 1000, submitted: 4000 of 4000)
	Chunk Failed (status code 400): 0 entities.
	Invalid records in this chunk: 0
Finished data submission.
Successful records: 4000
Failed invalid records: 0


In [38]:

print(f"time taken to push 810 recoords with sheepdog load_test_branch - {time_taken}")

#These are the values of the time taken to push 8100 records to the service
time_load_test_branch = 628.9082880020142  #But the chunk size was cut down to 500
time_load_test_branch_new_base = 641.9752421379089 #but service didn't fail after 1min, keeping chunk size at 1000
time_load_2501 = 299.92830085754395 # Benchnark time for 8100 records with tag 25.01


time taken to push 810 recoords with sheepdog load_test_branch - 442.25717210769653
